# Collaborative Filtering 
This section uses collaborative filtering techniques to find out top business a user might like based on his/her similarity to other users or based on similarity between the business we want to check and businesses he/she has liked

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import nltk
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
nltk.download('stopwords')
import pickle
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from collections import defaultdict

from surprise import SVD
from surprise import Dataset

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Reading the reviews as chunks since they all don't fit in memory.

In [3]:
reviews = pd.read_json("/data/yelp_academic_dataset_review.json", orient="records", lines=True, chunksize=20000)

In [4]:
for reviews in reviews:
    reviews = reviews[['review_id', 'user_id', 'business_id', 'stars']]
    break

In [5]:
reviews.describe()

,stars
count,20000.000000
mean,3.735850
std,1.443703
min,1.000000
25%,3.000000
50%,4.000000
75%,5.000000
max,5.000000


In [6]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   review_id    20000 non-null  object
 1   user_id      20000 non-null  object
 2   business_id  20000 non-null  object
 3   stars        20000 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 625.1+ KB


In [7]:
users_all = pd.read_json("/data/yelp_academic_dataset_user.json", chunksize=10000,orient="records", lines=True)

In [8]:
users = None
for e, chunk in enumerate(users_all):
    chunk = pd.merge(chunk, reviews, on='user_id', how='inner')
    if users is None:
        users = chunk
    else:
        users = pd.concat([users, chunk])
    if users.shape[0] >= 10000:
        break
    if e==1000:
        break

In [9]:
users.head()

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos,review_id,business_id,stars
0,q-v8elVPvKz0KvK69QSj1Q,Lisa Marie,666,2009-05-19 01:42:25,2993,1281,1832,"2011,2012,2013,2014,2015,2016,2017,2018","rt1KveqwFMnkN6dXKg5Qyg, NfnKx3z7zFottS3yHabw1g...",197,...,7,120,150,135,135,42,72,PaYbct-XdNPb3QoT0VzmmA,fgskuH5aQq0ROHm9zst_0g,4
1,Fta-vmTJYKGeVyyhm9D6Vw,Emily,3,2010-03-25 20:12:56,14,0,14,,"NMmu4oC-YRqdi0WY9pV78g, n5wQ7qm2QZY9iB8c25pzxA...",0,...,0,0,0,0,0,0,0,vcBDU6pbS08Xeba6W6HOcg,yNPh5SO-7wr8HPpVCDPbXQ,5
2,dIIKEfOgo0KqUfGQvGikPg,Gabi,2061,2007-08-10 19:01:51,20024,9684,16904,"2007,2008,2009,2010,2011,2012,2013,2014,2015,2...","6Y-l3x4LpUNhTBVMTFmTmA, HYNhRw_-8g660mpnwY2VJA...",971,...,96,1171,3272,2169,2169,463,281,BRTYuOLnAreHuyjtfU4rRQ,tDYcVluqZwieulc1iqxGXg,4
3,dIIKEfOgo0KqUfGQvGikPg,Gabi,2061,2007-08-10 19:01:51,20024,9684,16904,"2007,2008,2009,2010,2011,2012,2013,2014,2015,2...","6Y-l3x4LpUNhTBVMTFmTmA, HYNhRw_-8g660mpnwY2VJA...",971,...,96,1171,3272,2169,2169,463,281,qZjUgf2evwvTmKQYaF9EcA,5nZVVPO_cYH9aQeClTP9zA,3
4,dIIKEfOgo0KqUfGQvGikPg,Gabi,2061,2007-08-10 19:01:51,20024,9684,16904,"2007,2008,2009,2010,2011,2012,2013,2014,2015,2...","6Y-l3x4LpUNhTBVMTFmTmA, HYNhRw_-8g660mpnwY2VJA...",971,...,96,1171,3272,2169,2169,463,281,R6MVByeIwtW86OE4bQe25Q,zJGtD3y-pAIGNId4codEEg,4


In [10]:
users = users.drop(['name', 'review_count', 'yelping_since', 'useful', 'funny',
       'cool', 'elite', 'friends', 'fans', 'average_stars', 'compliment_hot',
       'compliment_more', 'compliment_profile', 'compliment_cute',
       'compliment_list', 'compliment_note', 'compliment_plain',
       'compliment_cool', 'compliment_funny', 'compliment_writer',
       'compliment_photos'], axis=1)

In [11]:
users.describe()

,stars
count,10826.000000
mean,3.784223
std,1.320996
min,1.000000
25%,3.000000
50%,4.000000
75%,5.000000
max,5.000000


In [13]:
businesses_all = pd.read_json("/data/yelp_academic_dataset_business.json", chunksize=10000,orient="records", lines=True)

In [14]:
businesses = None
for e, chunk in enumerate(businesses_all):
    chunk = pd.merge(chunk, reviews, on='business_id', how='inner')
    if businesses is None:
        businesses = chunk
    else:
        businesses = pd.concat([businesses, chunk])
    if businesses.shape[0] >= 10000:
        break
    if e==1000:
        break
    if e%10==0:
        print(e)

0


In [14]:
# businesses.head()

In [15]:
ind = businesses.business_id.isin(reviews.business_id) & reviews.user_id.isin(users.user_id)

In [16]:
ind.value_counts()

False    18945
True     10411
dtype: int64

In [17]:
ind = businesses.business_id.isin(users.business_id) & users.user_id.isin(businesses.user_id)
ind

0        True
0        True
0        True
0        True
0        True
        ...  
9661    False
9662    False
9663    False
9664    False
9665    False
Length: 37254, dtype: bool

In [18]:
ind.value_counts()

False    19160
True     18094
dtype: int64

In [19]:
businesses.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars_x,review_count,is_open,attributes,categories,hours,review_id,user_id,stars_y
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,28031,35.462724,-80.852612,3.5,36,1,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...","{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0'...",YkpjR5N5TLkwHi20IZ15zg,xDtS2iKsJuVUVzB2YhfPsg,4
1,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,4827 E Downing Cir,Mesa,AZ,85205,33.428065,-111.726648,4.5,26,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Home Services, Plumbing, Electricians, Handyma...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-16:0', '...",EY8t3ndAZo2vWY7eeOnVLw,nKBtfZ93gPYybGEz2QOvTQ,5
2,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,4827 E Downing Cir,Mesa,AZ,85205,33.428065,-111.726648,4.5,26,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Home Services, Plumbing, Electricians, Handyma...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-16:0', '...",uoujAVvyx-GLyQnByuon0w,UjVtviHTm2mgZnXCfl33CQ,5
3,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,4827 E Downing Cir,Mesa,AZ,85205,33.428065,-111.726648,4.5,26,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Home Services, Plumbing, Electricians, Handyma...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-16:0', '...",pECK3p9w7m-_xEp--lGxHg,L498DJb5YDAtoqgv9thWCg,5
4,cKyLV5oWZJ2NudWgqs8VZw,Oasis Auto Center - Gilbert,"1720 W Elliot Rd, Ste 105",Gilbert,AZ,85233,33.350399,-111.827142,4.5,38,1,{'BusinessAcceptsCreditCards': 'True'},"Auto Repair, Automotive, Oil Change Stations, ...","{'Monday': '7:0-18:0', 'Tuesday': '7:0-18:0', ...",edUqROoq7qAVzR2b27j2dA,4oE6BjeHyTVRqd_82LcrhQ,2


In [20]:
users.shape

(10826, 4)

In [21]:
businesses.shape

(19022, 17)

In [22]:
users[users['user_id'].isin(businesses.user_id)].shape

(10392, 4)

In [23]:
businesses[businesses['business_id'].isin(users.business_id)].shape

(16512, 17)

In [24]:
businesses.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars_x', 'review_count', 'is_open',
       'attributes', 'categories', 'hours', 'review_id', 'user_id', 'stars_y'],
      dtype='object')

In [25]:
businesses=businesses[['user_id', 'business_id', 'stars_y']]
businesses.head()

,user_id,business_id,stars_y
0,xDtS2iKsJuVUVzB2YhfPsg,f9NumwFMBDn751xgFiRbNA,4
1,nKBtfZ93gPYybGEz2QOvTQ,51M2Kk903DFYI6gnB5I6SQ,5
2,UjVtviHTm2mgZnXCfl33CQ,51M2Kk903DFYI6gnB5I6SQ,5
3,L498DJb5YDAtoqgv9thWCg,51M2Kk903DFYI6gnB5I6SQ,5
4,4oE6BjeHyTVRqd_82LcrhQ,cKyLV5oWZJ2NudWgqs8VZw,2


In [26]:
businesses.columns = ['user_id', 'business_id', 'stars']
businesses.head()

,user_id,business_id,stars
0,xDtS2iKsJuVUVzB2YhfPsg,f9NumwFMBDn751xgFiRbNA,4
1,nKBtfZ93gPYybGEz2QOvTQ,51M2Kk903DFYI6gnB5I6SQ,5
2,UjVtviHTm2mgZnXCfl33CQ,51M2Kk903DFYI6gnB5I6SQ,5
3,L498DJb5YDAtoqgv9thWCg,51M2Kk903DFYI6gnB5I6SQ,5
4,4oE6BjeHyTVRqd_82LcrhQ,cKyLV5oWZJ2NudWgqs8VZw,2


In [27]:
users = users[['user_id', 'business_id', 'stars']]
users.head()

,user_id,business_id,stars
0,q-v8elVPvKz0KvK69QSj1Q,fgskuH5aQq0ROHm9zst_0g,4
1,Fta-vmTJYKGeVyyhm9D6Vw,yNPh5SO-7wr8HPpVCDPbXQ,5
2,dIIKEfOgo0KqUfGQvGikPg,tDYcVluqZwieulc1iqxGXg,4
3,dIIKEfOgo0KqUfGQvGikPg,5nZVVPO_cYH9aQeClTP9zA,3
4,dIIKEfOgo0KqUfGQvGikPg,zJGtD3y-pAIGNId4codEEg,4


In [28]:
users.shape

(10826, 3)

In [29]:
businesses.shape

(19022, 3)

In [30]:
intersection = users.merge(businesses, how = 'inner' ,on = ['user_id', 'business_id'])
intersection.head()

,user_id,business_id,stars_x,stars_y
0,q-v8elVPvKz0KvK69QSj1Q,fgskuH5aQq0ROHm9zst_0g,4,4
1,Fta-vmTJYKGeVyyhm9D6Vw,yNPh5SO-7wr8HPpVCDPbXQ,5,5
2,dIIKEfOgo0KqUfGQvGikPg,tDYcVluqZwieulc1iqxGXg,4,4
3,dIIKEfOgo0KqUfGQvGikPg,5nZVVPO_cYH9aQeClTP9zA,3,3
4,dIIKEfOgo0KqUfGQvGikPg,zJGtD3y-pAIGNId4codEEg,4,4


In [31]:
intersection.shape

(10327, 4)

In [32]:
union = users.merge(businesses, how = 'outer')
union.head()

,user_id,business_id,stars
0,q-v8elVPvKz0KvK69QSj1Q,fgskuH5aQq0ROHm9zst_0g,4
1,Fta-vmTJYKGeVyyhm9D6Vw,yNPh5SO-7wr8HPpVCDPbXQ,5
2,dIIKEfOgo0KqUfGQvGikPg,tDYcVluqZwieulc1iqxGXg,4
3,dIIKEfOgo0KqUfGQvGikPg,5nZVVPO_cYH9aQeClTP9zA,3
4,dIIKEfOgo0KqUfGQvGikPg,zJGtD3y-pAIGNId4codEEg,4


In [33]:
union.shape

(19569, 3)

In [34]:
union = union.drop_duplicates()
union.shape

(19536, 3)

## Preparing dataset
We use the surprise algorithm for recommendations, so we prepare the data using the dataframe we created.

In [35]:
import pandas as pd
from surprise import Dataset
from surprise import Reader
from surprise import KNNWithMeans


reader = Reader(rating_scale=(1, 5))

# Loads Pandas dataframe
data = Dataset.load_from_df(union[["user_id", "business_id", "stars"]], reader)
# create training/test set
trainingSet = data.build_full_trainset()
# users that are not included in the training set
testset = trainingSet.build_anti_testset()


In [36]:
# users that ARE included in the training set (for validation)
testset2 = trainingSet.build_testset()

In [37]:
print(trainingSet.n_items, trainingSet.n_users, trainingSet.n_ratings)

6354 17955 19536


In [38]:
print(len(testset), len(testset2))

114066546 19536


In [39]:
intersection.head()

,user_id,business_id,stars_x,stars_y
0,q-v8elVPvKz0KvK69QSj1Q,fgskuH5aQq0ROHm9zst_0g,4,4
1,Fta-vmTJYKGeVyyhm9D6Vw,yNPh5SO-7wr8HPpVCDPbXQ,5,5
2,dIIKEfOgo0KqUfGQvGikPg,tDYcVluqZwieulc1iqxGXg,4,4
3,dIIKEfOgo0KqUfGQvGikPg,5nZVVPO_cYH9aQeClTP9zA,3,3
4,dIIKEfOgo0KqUfGQvGikPg,zJGtD3y-pAIGNId4codEEg,4,4


In [40]:
trainingSet.to_inner_uid('dIIKEfOgo0KqUfGQvGikPg')

2

0

In [41]:
testset[:20]

[('q-v8elVPvKz0KvK69QSj1Q', 'yNPh5SO-7wr8HPpVCDPbXQ', 3.740223177723178),
 ('q-v8elVPvKz0KvK69QSj1Q', 'tDYcVluqZwieulc1iqxGXg', 3.740223177723178),
 ('q-v8elVPvKz0KvK69QSj1Q', '5nZVVPO_cYH9aQeClTP9zA', 3.740223177723178),
 ('q-v8elVPvKz0KvK69QSj1Q', 'zJGtD3y-pAIGNId4codEEg', 3.740223177723178),
 ('q-v8elVPvKz0KvK69QSj1Q', 'TShoU62Hm-4wH2EzwkJbrg', 3.740223177723178),
 ('q-v8elVPvKz0KvK69QSj1Q', 'QeGkAHAKUT-TkyPUsl2PIg', 3.740223177723178),
 ('q-v8elVPvKz0KvK69QSj1Q', 'nY2oBwH3s5sKox1mERIoVQ', 3.740223177723178),
 ('q-v8elVPvKz0KvK69QSj1Q', 'ez68a9bacyFKNy6MG6_kRg', 3.740223177723178),
 ('q-v8elVPvKz0KvK69QSj1Q', 'Cy9LmQrY9rWwaNTSqxQn6A', 3.740223177723178),
 ('q-v8elVPvKz0KvK69QSj1Q', 'BGGQOJQTQerEQu0kHbT_UQ', 3.740223177723178),
 ('q-v8elVPvKz0KvK69QSj1Q', 'XnJeadLrlj9AZB8qSdIR2Q', 3.740223177723178),
 ('q-v8elVPvKz0KvK69QSj1Q', '_ZfjpSEO5ntk-1hbnwCR4g', 3.740223177723178),
 ('q-v8elVPvKz0KvK69QSj1Q', 'NEVA0IYbawceL6kz5v5DAw', 3.740223177723178),
 ('q-v8elVPvKz0KvK69QSj1Q', 'nnDLapJk1

## Recommender systems using nearest neighbor
We try two different approaches. The first one is an item based approach. For a given business to the user, the algorithm checks all other businesses the user has rated and makes prediction based on top most similar businesses. 

The second one is a user-based approach. For a given business to the user, the algorithm checks all other users who have rated this business and calculates their similarities to the user. The prediction is made based on the top most similar users to the the user in question.


In [42]:

sim_options = {
    "name": "cosine",
    "user_based": False,  
}
item_based_recommender = KNNWithMeans(sim_options=sim_options)

sim_options = {
    "name": "cosine",
    "user_based": True, 
}
user_based_recommender = KNNWithMeans(sim_options=sim_options)

In [43]:
item_based_recommender.fit(trainingSet)
user_based_recommender.fit(trainingSet)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [44]:
from surprise import accuracy

ub_predictions = user_based_recommender.test(testset2)
ib_predictions = item_based_recommender.test(testset2)

print('MSE by user based: {} \t MSE by item based: {}'.format(accuracy.mse(ub_predictions), accuracy.mse(ib_predictions)))
print('RMSE by user based: {} \t RMSE by item based: {}'.format(accuracy.rmse(ub_predictions), accuracy.rmse(ib_predictions)))
print('MAE by user based: {} \t MAE by item based: {}'.format(accuracy.mae(ub_predictions), accuracy.mae(ib_predictions)))

MSE: 0.0603
MSE: 0.0577
MSE by user based: 0.06029027199114488 	 MSE by item based: 0.05769530740166643
RMSE: 0.2455
RMSE: 0.2402
RMSE by user based: 0.2455407745999529 	 RMSE by item based: 0.24019847501944394
MAE:  0.0822
MAE:  0.0648
MAE by user based: 0.08223940508140339 	 MAE by item based: 0.06480219541912444


In [45]:
# taken from: https://surprise.readthedocs.io/en/stable/FAQ.html
def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [46]:
unique = list(set(testset[:10000]))

In [47]:
ub_predictions = user_based_recommender.test(unique[:10])
ib_predictions = item_based_recommender.test(unique[:10])

top_n_ub = get_top_n(ub_predictions, n=10)
top_n_ib = get_top_n(ib_predictions, n=10)



# Print the recommended items for each user
for uid, user_ratings in top_n_ub.items():
    print(uid, ':')
    for i, (iid, score) in enumerate(user_ratings):
        print('\tUser Based: {} {} \t Item Based: {} {}'.format(iid, score, top_n_ib[uid][i][0], top_n_ib[uid][i][1]))
#         print('\t', iid, top_n_ib[uid][i][0])


Fta-vmTJYKGeVyyhm9D6Vw :
	User Based: j2OuzaFMJHQXjVHy_3bNgA 5 	 Item Based: 7uGsMlo8_uIZIITKfhEARA 3.6666666666666665
	User Based: 7uGsMlo8_uIZIITKfhEARA 5 	 Item Based: d4qwVw4PcN-_2mK2o1Ro1g 3.5
	User Based: d4qwVw4PcN-_2mK2o1Ro1g 5 	 Item Based: j2OuzaFMJHQXjVHy_3bNgA 1
q-v8elVPvKz0KvK69QSj1Q :
	User Based: OD2hnuuTJI9uotcKycxg1A 4.0 	 Item Based: W_9ophRRWO924hn1S-y2Hg 5
	User Based: W_9ophRRWO924hn1S-y2Hg 4.0 	 Item Based: D2D2qsxifHaRqNlVhP3YTA 5
	User Based: D2D2qsxifHaRqNlVhP3YTA 4.0 	 Item Based: sOQBfSKH9EXM6nRgtGaE3w 5
	User Based: sOQBfSKH9EXM6nRgtGaE3w 4.0 	 Item Based: Cuo9w-g5Z26d0-CsaPDGuA 5
	User Based: Cuo9w-g5Z26d0-CsaPDGuA 4.0 	 Item Based: 9LE2yGl23qxitNc9mkpbjA 4.0
	User Based: 9LE2yGl23qxitNc9mkpbjA 4.0 	 Item Based: s38vKr-DGXMXuTkUic2qoA 2.5
	User Based: s38vKr-DGXMXuTkUic2qoA 4.0 	 Item Based: OD2hnuuTJI9uotcKycxg1A 1


In [48]:
unique = list(set(testset2[:10000]))
ub_predictions = user_based_recommender.test(unique[:10])
ib_predictions = item_based_recommender.test(unique[:10])

top_n_ub = get_top_n(ub_predictions, n=10)
top_n_ib = get_top_n(ib_predictions, n=10)



# Print the recommended items for each user
for uid, user_ratings in top_n_ub.items():
    print(uid, ':')
    for i, (iid, score) in enumerate(user_ratings):
        print('\tUser Based: {} {} \t Item Based: {} {}'.format(iid, score, top_n_ib[uid][i][0], top_n_ib[uid][i][1]))
#         print('\t', iid, top_n_ib[uid][i][0])


bOkTNMHm03jfWbqqrDCYvA :
	User Based: xsFeeqZS6PoVvwseEefzEw 2.9166666666666665 	 Item Based: xsFeeqZS6PoVvwseEefzEw 3.0
CdqP6XZM-Dl3NxjWLqoeZA :
	User Based: bjJcDCx5uMUr89nxBdGizw 5 	 Item Based: bjJcDCx5uMUr89nxBdGizw 5
Kez5MabViniF-HUZ166HFw :
	User Based: FjL-V4kj2jHeoByEwjK5Eg 5 	 Item Based: FjL-V4kj2jHeoByEwjK5Eg 5
HpuTSKKxY-7S8-ycWuI7rw :
	User Based: -BbnAc9YEO6pjvJGEtFbVQ 4.115384615384615 	 Item Based: -BbnAc9YEO6pjvJGEtFbVQ 4.0
o5u6UMuVgJcl_oJLe44afw :
	User Based: buGZ95frO6OTrRhFxDcFQg 4.0 	 Item Based: buGZ95frO6OTrRhFxDcFQg 4.0
F-Uyh6JypKNy0aK-8Hc6hQ :
	User Based: Jz4LZvTibMEzXx8nZX5Q7g 4.0 	 Item Based: Jz4LZvTibMEzXx8nZX5Q7g 4.0
cYyOz1Z2S3smsovo2Vl2nQ :
	User Based: 5JW08_RL9K60Ubo8-8lyVg 5 	 Item Based: 5JW08_RL9K60Ubo8-8lyVg 5
bhmJrMRVc2sbk8XP00ravw :
	User Based: buvPYzg6MsOsHBMy6iWHoQ 3.666666666666667 	 Item Based: buvPYzg6MsOsHBMy6iWHoQ 4.243055555555555
ciXjBfJrAEteIKpzZg4I9g :
	User Based: QFqtVX-lyZQgVHQrG-dhlg 3.3425925925925926 	 Item Based: QFqtVX-lyZQgV

In [83]:
reviews = pd.read_json("/data/yelp_academic_dataset_review.json", orient="records", lines=True, chunksize=1000)


In [105]:
user_ids = union['user_id'].tolist()
business_ids = union['business_id'].tolist()
uid_not_bid = []
bid_not_uid = []

for u in user_ids:
    try:
        uid_int = trainingSet.to_inner_uid(u)
        print(u, trainingSet.knows_user(uid_int))
        try:
            bid = union[union['user_id']==u]['business_id'].item()
            bid_int = trainingSet.to_inner_iid(bid)
        except:
            print(u, 'bid for this user is not in trainset')
            uid_not_bid.append((u, bid))               
    except:
        print(u, 'user is not in trainset')

        

q-v8elVPvKz0KvK69QSj1Q True
Fta-vmTJYKGeVyyhm9D6Vw True
dIIKEfOgo0KqUfGQvGikPg True
dIIKEfOgo0KqUfGQvGikPg bid for this user is not in trainset
dIIKEfOgo0KqUfGQvGikPg True
dIIKEfOgo0KqUfGQvGikPg bid for this user is not in trainset
dIIKEfOgo0KqUfGQvGikPg True
dIIKEfOgo0KqUfGQvGikPg bid for this user is not in trainset
dIIKEfOgo0KqUfGQvGikPg True
dIIKEfOgo0KqUfGQvGikPg bid for this user is not in trainset
dIIKEfOgo0KqUfGQvGikPg True
dIIKEfOgo0KqUfGQvGikPg bid for this user is not in trainset
JnPIjvC0cmooNDfsa9BmXg True
JnPIjvC0cmooNDfsa9BmXg bid for this user is not in trainset
JnPIjvC0cmooNDfsa9BmXg True
JnPIjvC0cmooNDfsa9BmXg bid for this user is not in trainset
JnPIjvC0cmooNDfsa9BmXg True
JnPIjvC0cmooNDfsa9BmXg bid for this user is not in trainset
o-GAkbTTcHFY3KqVDCE8sA True
mAhty2OHDTtLm00quB5Xjw True
mAhty2OHDTtLm00quB5Xjw bid for this user is not in trainset
mAhty2OHDTtLm00quB5Xjw True
mAhty2OHDTtLm00quB5Xjw bid for this user is not in trainset
4mjnkd8oJVCfBKN3i4rB-g True
sR3vq6nT

hBicl52tMVsTecdZSVL-lQ True
Q58Dk9jLWLCr2m6Ip08T0w True
tQVTyovdm4doXxDSkfYUUw True
4ZfcCa4m5RWvO4EFzfYm1A True
4ZfcCa4m5RWvO4EFzfYm1A bid for this user is not in trainset
4ZfcCa4m5RWvO4EFzfYm1A True
4ZfcCa4m5RWvO4EFzfYm1A bid for this user is not in trainset
9hMqNBBpTl3lBy6qCAyu-A True
9hMqNBBpTl3lBy6qCAyu-A bid for this user is not in trainset
9hMqNBBpTl3lBy6qCAyu-A True
9hMqNBBpTl3lBy6qCAyu-A bid for this user is not in trainset
tFTU5yWOCQLqJMs2YhAaFQ True
DKdttAvXagfAfNFIKeApuw True
DKdttAvXagfAfNFIKeApuw bid for this user is not in trainset
DKdttAvXagfAfNFIKeApuw True
DKdttAvXagfAfNFIKeApuw bid for this user is not in trainset
8msyzcHj92LGxqIf8LoOVw True
tBldrCI2KDA_WfYNloHqsA True
dvgzLpLii0MOlR-X59MlGg True
-hKniZN2OdshWLHYuj21jQ True
9s0619fxzS6M-X4pVS7W_A True
CZ84s3T6iysXAX10CfH2ZA True
bhJ6ivAuSpgXP4JiKWjSZQ True
bhJ6ivAuSpgXP4JiKWjSZQ bid for this user is not in trainset
bhJ6ivAuSpgXP4JiKWjSZQ True
bhJ6ivAuSpgXP4JiKWjSZQ bid for this user is not in trainset
bhJ6ivAuSpgXP4Ji

maK3UBQczh33NuDjBYeHrA True
ANJ-jY5M8k-FKd7kr7x93w True
5HSrzwZlEJpsDyO83B4EQA True
I9oGYCN2ypUkzd078jFyEw True
pNSz6v7QCYNWr6Mz0gBj9Q True
6WfBop_XS1EgHU3F3ACqKA True
-RhRXVW9z9fs5zzxhFfnHg True
ASB-bnmB6wta1c7N3VQsTg True
AuIK5tF2GjO7SftHawTLKw True
GLjJ1WZDmVuCguPr7mKw5Q True
EWQil1sWe23p9uoEU5AZlg True
gUkqdzdD0jhqsjM-AZAWVw True
uqJ0kCrmZOEZzbt4qV3ODA True
QS8hvLT-Ks3gHDZBrz8D7w True
PeLGa5vUR8_mcsn-fn42Jg True
PeLGa5vUR8_mcsn-fn42Jg bid for this user is not in trainset
PeLGa5vUR8_mcsn-fn42Jg True
PeLGa5vUR8_mcsn-fn42Jg bid for this user is not in trainset
PeLGa5vUR8_mcsn-fn42Jg True
PeLGa5vUR8_mcsn-fn42Jg bid for this user is not in trainset
PeLGa5vUR8_mcsn-fn42Jg True
PeLGa5vUR8_mcsn-fn42Jg bid for this user is not in trainset
psSJdH2V1fNlwfuAk4D7KQ True
JRn2ZEQGrUICoARCKJogBA True
Oi-JGR5OK2qxedFuH0lpBQ True
XKvbfNylF6Yq8hpwqaZrhA True
M1mSvucWF3V1h0L98okb9g True
dipVxaat-2-XN3OHWcZjmg True
Xxvz5g67eaCr3emnkY5M6w True
0UI9KTPdtkwl9lMdMT-Sgg True
aKQ9RToYxsRQATnf5sBTQA True
aGI_

43rpaoRriVutxQbkSxe6pA bid for this user is not in trainset
43rpaoRriVutxQbkSxe6pA True
43rpaoRriVutxQbkSxe6pA bid for this user is not in trainset
LnsstbXJSOTc0dJOC2sPug True
jJsEURuPmW059FAKHZQvvw True
w7Q-WMxPuznP-P2Y79ICng True
3IOBf3pKHwxF3NO6k6NA9Q True
_txhy3UvyGffFh8FLO2_qw True
coW_WMRPEqZiLds9FtNzCw True
fqCxaOI1cvnXx1OlHRUDPg True
lOFBy0gVGOV0VapjoNZdlQ True
ierqPx-n-6fr6gHF2gNUqQ True
iAevEfcmtCr2cpd6IRqnTA True
DfOj2Jf4MlTQiF5TMHCm2g True
96aWRa-gy1RrsrFQURagvg True
nmdlcC7Vk-fnhMAre7Tz_Q True
jlpapIPWURjBUg3V31AjKw True
Ox0Nic-2-6pb7V4jyi9iFA True
i8hCMZN-0bHENsHZKHpC-g True
QjEHA4t7BovzKVeH8TzT2A True
QjEHA4t7BovzKVeH8TzT2A bid for this user is not in trainset
QjEHA4t7BovzKVeH8TzT2A True
QjEHA4t7BovzKVeH8TzT2A bid for this user is not in trainset
-C74IbRL4LAEfb6jXIG_gg True
53EWBX2QGISTDZJ1NZfyrA True
XO0UfcfbYOrcjtggHeZN2w True
h0nPx7G9gvdhVPLZptTy4g True
2xp2PStd-7u53jh30DkC3A True
YwaKGmRNnSa3R3N4Hf9jLw True
AM3NGe9wtZDiwUyB31DZ2g True
CxDOIDnH8gp9KXzpBHJYXw True
CxDO

mySDXsBhY3u0yI-G2oVDkA True
STcrjP1twU03Vk6syLft7g True
STcrjP1twU03Vk6syLft7g bid for this user is not in trainset
STcrjP1twU03Vk6syLft7g True
STcrjP1twU03Vk6syLft7g bid for this user is not in trainset
l1blKe-Yz1ksJWuR94x5SA True
B7ecAeAIrXg7sgmabS38pg True
B7ecAeAIrXg7sgmabS38pg bid for this user is not in trainset
B7ecAeAIrXg7sgmabS38pg True
B7ecAeAIrXg7sgmabS38pg bid for this user is not in trainset
-VtIPsp0TgYyO4bg9vyG9A True
LLsyjn2GZ715AwIPlWZ4Pg True
LLsyjn2GZ715AwIPlWZ4Pg bid for this user is not in trainset
LLsyjn2GZ715AwIPlWZ4Pg True
LLsyjn2GZ715AwIPlWZ4Pg bid for this user is not in trainset
zzPvEodjvLPe-5NvV0U_cg True
YcaZNr4HQ5f3ihajDQQLJw True
_syH6ryl48-UFYL5KsXqZA True
DpiMuxg2g4urzClzjykUHg True
I8K8inWlqoQ3af_CAER56w True
trV3UzJZi2vUSMVrHaDIFg True
A0Tkq4VV8dD6t8NbBxvBPA True
A0Tkq4VV8dD6t8NbBxvBPA bid for this user is not in trainset
A0Tkq4VV8dD6t8NbBxvBPA True
A0Tkq4VV8dD6t8NbBxvBPA bid for this user is not in trainset
A0Tkq4VV8dD6t8NbBxvBPA True
A0Tkq4VV8dD6t8Nb

...

xIk0vd0WFtQ2VJGRdtxIHw True
4X8OUXV9xUNKasKb006vWg True
h9zkDOsXqDZsCiS_fKMr6w True
xstHNJxOCdV1lLJDsKKLGA True
48eEHoEp9N-nZMZEt5aMYg True
zSqMbeNZXI-qippJ_vLmfw True
l7eXX1a_mAkzPc09-l4Vtw True
9LxrQZ1HIlzrArvaHmGazg True
oSwenE7WfyUztPgJGX7ynQ True
wFG2bellkz7PtnE9I6Or5w True
AmK92e8UaGUfYII42I2-Kw True
Bo-9m6oPR2eSZPBVLTW47Q True
GYRsuFF4SIgeUIYZlQDkPQ True
JUcoCpepo4X5TblImudH1g True
M9VczfEvbKwAkwZZpM1hvg True
anji5i3py5OslgB-z6MirQ True
iZW2fqYcWCFswl0tpwNpQQ True
0IwZb1ePNpSVYcCdMtGoQw True
SPPieDvwnkvQoI7lj5vFYg True
wNtRrYCn5F5A6kFcs-gqGw True
T_IAPiKji8_GSkgQ6ziZ4g True
wCnNjdGQ-Oc6cVFdmIJaDw True
nywsayLLsFWK42pitFDitw True
x9p0RQsdWD-NZuhH4J1zHw True
x9p0RQsdWD-NZuhH4J1zHw bid for this user is not in trainset
gMRRLOmwlifjZb_-nC7XEw True
E99kH7Vvbdu24P1lDx1vrQ True
tC6edtSfHq4k5KW85ea2iA True
OjEe11tYUiIX8Y_fSQX8uA True
Ez-Cxj-6IXjdA_5pWXOIqQ True
fY3Z_ecDAM4gwRNHbDuMeg True
383UDZYlbY9gaKNo7Dv9UA True
mWkNQSUTcD7skktw225Pwg True
-nzpoMa5ahDemykxo77-Bw True
RR-2nouBn408e3dj

s53RByNJWveOJ6NBKG9I3A True
FX73lAFOt9BUBEN-Qh4RvA True
b9ovcVsSr1VsS8fzrD_jrw True
XkLRbSi-5xYBTBRIivIHhg True
nzv6DRL_GN68GydZHjPlyg True
0q2FtT-tMXI4JCJMn6YA4w True
FXtikKFqF6eyJdPHdk0miQ True
JgKO1d7XiG1iqBNw3t0ptA True
WrIG89EHAf6jEf3tbX_DJQ True
WrIG89EHAf6jEf3tbX_DJQ bid for this user is not in trainset
hCSA-jHrYaVp7Go09eM6yA True
e3a9FYT71hfjj8YI80KvIQ True
AC7VDjPYudKUT13BVd0VXg True
AC7VDjPYudKUT13BVd0VXg bid for this user is not in trainset
LNgTVGyf2jfBssdeo8sT8w True
SLTHacR_qZGlDLx7lmleEA True
LzYwHinpbHk3YQrIkVI4KQ True
jCuD5qF7faKsdiXJ5Ow-mg True
CVz59fHlXSphe2WtOUvo0Q True
VLEgZtFm9JLOhq4D3hA4Ew True
NLDaC-HeRaGIDBz5afntag True
kt3J-u1s6yLgsObSfC68mg True
5TZhuQxi25pecT8e7ASd-w True
oe1Y05n3dI3FCFpz4AtkbQ True
8ec8Wuw_zC7PLE4Mkd_9Ig True
8ec8Wuw_zC7PLE4Mkd_9Ig bid for this user is not in trainset
S4z-xLmyI2dGmGmsGYL0jg True
07KAbICC4m8B2_x8bxAGHQ True
m6T3lhzu2DLF5oIR7xxLjg True
zR2RIA8HL2YoRwuwy-4E6g True
zR2RIA8HL2YoRwuwy-4E6g bid for this user is not in trainset
Pc4L

Z0aJ19iOm3U__lBEbLTbzg True
1Bm275hGxUml7HlnwJu3oQ True
i_YJHmRYP1jxWdHKsRueQw True
HfsENIwk3K816glyfsTyqQ True
rOodSOypj4os4xooirRJfQ True
1MHy9cMltOhEycJ6SmQyfw True
ySACP4uUD7sQzHkT2yJwxg True
o4URejXdIzcGNg8Zohn0GQ True
oYrKH2Now-qfaUCxpgaBwg True
_vlJO3S8s0VQRfneK_-NyQ True
TcgES9Hrsz4g8EIBvXqWvQ True
QyKtK6hFfanlucU4xEv0WA True
RRU7Pam9r7VSinQfQs_X6g True
7SVJlB6fGzZZdO2C2pJ9JQ True
6Vr5hJXLj0WhiZJgIfhzOQ True
cDpVoZb35QcBfhtPJL7X3w True
nDFtPJfB2iq8yeGX39SpKA True
FBj6OZ5ks9wLmRJOAeGoXA True
lPIS5_4APNUY0R0-oJ4_8A True
uOeaKopmw70mjTRajDxV8Q True
hx-3j1EYydQURSkGajFt5A True
-VggqkdCd5ny0Fl3UDqeKA True
UIJMQU0TSEeOf4ntpTzu1w True
s5BG_ObtewqCCVfCWiPk_g True
8z_EK_OYXGOR4cd5qiEBHw True
MdXA5C87R7gRBHPz6_-Wgg True
lVRXEOkdd6-xUr9qHLiH6w True
G-2oM9-NS6X-LXX1GEbuOA True
yS1kK5EGcwatp6D6hFGvlw True
tLl6nrX280K-7crTpeTwTw True
tLl6nrX280K-7crTpeTwTw bid for this user is not in trainset
z3fa-k0-RTH_K7c4gAEYPA True
PZCEYFrSXjQXTmDpPUwrWw True
vpvW58vXP9xuOsWg-OCFxg True
94YwuBX_cJyH6tzz

BEpNYLOIQPaC6QJsnw1Lbw True
obnF_WhGTFn0zp2OGK66rQ True
yrsqWFzvu4RTXGKoA-uG1g True
Yw5LynmZmKjSb4cuzgHptw True
Yw5LynmZmKjSb4cuzgHptw bid for this user is not in trainset
5ufXAWf50X4IlORgHZOK3g True
T494zLm9Au-pHIALtLv_hw True
NHSDbjVukgqnKXULUFUCnQ True
yrTZ3Z_k-bYS4q5izF8d5w True
zDKszv6zpuLsVhPFZLctXQ True
n0Y7qySGyhUVh1fjmgWG7Q True
oIdFHJdGu1rP6PtYiWW5IQ True
yFqiFZGKEW9lWhgUkPqSXA True
Ca0JY-f3V9YuBuJHEtGKMA True
p14_fU99YOP1GtmTu2d-KQ True
p14_fU99YOP1GtmTu2d-KQ bid for this user is not in trainset
XLuqR7EZYnDEek74fGUliQ True
MJrlfAjtGhZ9-fE370Cw_w True
l44tgWB_Kl0CCmfEfZgeZA True
9RX9FRHIBCTUyLgbM9_fBg True
z5Kgqa7ySVR-YdgM-wXAxQ True
agGWDhkVvIHaf8w16jLTng True
SScNU-2S-SLpiJFrIAphew True
SScNU-2S-SLpiJFrIAphew bid for this user is not in trainset
h4NoXg-ejCe3dzJjAHyK2Q True
qLgnAMqJChFjHGEZdyyznw True
yCu9vnBX46TID6HskgtCtQ True
Fsoi41eQf1IXpYaCpv693A True
Fsoi41eQf1IXpYaCpv693A bid for this user is not in trainset
CrXTuKbN9EtNtNdwBNIbsQ True
iwF0TZv3Ks5FjGsFJV2AtA True
f7fu

rYhhwBBPmrV9UwN7VIrnTQ True
xCdShNSUgXrF2sAZas4f1w True
xCdShNSUgXrF2sAZas4f1w bid for this user is not in trainset
leAgIlXp5Ll1F4_DglbAUg True
edjpVqeAjbWGKdGqbYWjTQ True
ymaFC3h-gqHwtzVznQw5JQ True
wfaFHpc9Wp-2ltPqUZbn1Q True
NUeajZ3JMmZ4W-N1sxT8Dg True
NIUofZwMISQp-FAk3yKihg True
__tZI6uEWpbiGpLADZQEuA True
gScOCEV-Wz-2s0pkTrWCKA True
RYa1fWYn7B_bz_HGNXlvsQ True
BmjCPGZ2d_4ePzuYDOD-Wg True
zpxbqoF0AJwVPOxmD5jSEQ True
Plj_EalKK47zKFQzNbfrFQ True
Y3qVdeES0gDNxUgbAFg0sg True
ApTc8z06CFZL6zLS_zN1Vw True
XYl43Jgcl_AgM5YXDFVy5A True
XYl43Jgcl_AgM5YXDFVy5A bid for this user is not in trainset
rJgOiLlUFC-kHfKiDy_NfQ True
o7SkST5tUUL09yUB8_tQ-A True
iDfQikdiXnhu03VBFM4rqg True
tbz6pWj4rh35pO1WFrGHuw True
pOG2MEYn3NACqlIjmSlZ7Q True
Gva6kPW-ia3bUhfggOYcmQ True
UlCvHXwMjtQ_6y0qTmuBYQ True
NI299hVk072RaOCL1NDBog True
MYPdEElYeiZ5x3vUryqB1Q True
aNsvWp5-qnwlvlSV-57MCw True
hYZH2pMSPZNwelhg_C_UbA True
xpjDl54w1y3xvLohkGKNRw True
xpjDl54w1y3xvLohkGKNRw bid for this user is not in trainset
vKGVOBjz

In [106]:
for b in business_ids:
    try:
        bid_int = trainingSet.to_inner_iid(b)
        print(b, trainingSet.knows_item(bid_int))
        try:
            uid = union[union['business_id']==b]['user_id'].item()
            uid_int = trainingSet.to_inner_uid(uid)
        except:
            print(b, 'uid for this business is not in trainset')
            bid_not_uid.append((uid, b))               
    except:
        print(b, 'business is not in trainset')

fgskuH5aQq0ROHm9zst_0g True
fgskuH5aQq0ROHm9zst_0g uid for this business is not in trainset
yNPh5SO-7wr8HPpVCDPbXQ True
yNPh5SO-7wr8HPpVCDPbXQ uid for this business is not in trainset
tDYcVluqZwieulc1iqxGXg True
tDYcVluqZwieulc1iqxGXg uid for this business is not in trainset
5nZVVPO_cYH9aQeClTP9zA True
5nZVVPO_cYH9aQeClTP9zA uid for this business is not in trainset
zJGtD3y-pAIGNId4codEEg True
zJGtD3y-pAIGNId4codEEg uid for this business is not in trainset
TShoU62Hm-4wH2EzwkJbrg True
TShoU62Hm-4wH2EzwkJbrg uid for this business is not in trainset
QeGkAHAKUT-TkyPUsl2PIg True
QeGkAHAKUT-TkyPUsl2PIg uid for this business is not in trainset
nY2oBwH3s5sKox1mERIoVQ True
nY2oBwH3s5sKox1mERIoVQ uid for this business is not in trainset
ez68a9bacyFKNy6MG6_kRg True
Cy9LmQrY9rWwaNTSqxQn6A True
BGGQOJQTQerEQu0kHbT_UQ True
BGGQOJQTQerEQu0kHbT_UQ uid for this business is not in trainset
XnJeadLrlj9AZB8qSdIR2Q True
XnJeadLrlj9AZB8qSdIR2Q uid for this business is not in trainset
_ZfjpSEO5ntk-1hbnwCR4g T

N-FKBizx_wu3L8mvDPfYGw True
N-FKBizx_wu3L8mvDPfYGw uid for this business is not in trainset
vJmD1dagBjER83Raf2KAWQ True
vJmD1dagBjER83Raf2KAWQ uid for this business is not in trainset
StjUke9uj6T6qB4hpyzMMA True
StjUke9uj6T6qB4hpyzMMA uid for this business is not in trainset
KsWGy-WuTUm5ve5lFUOCfg True
KsWGy-WuTUm5ve5lFUOCfg uid for this business is not in trainset
Rj3wnYT1QAxpJlRT5mMc-g True
Rj3wnYT1QAxpJlRT5mMc-g uid for this business is not in trainset
UCpUOtvqR-NBWBNVMzJleA True
UCpUOtvqR-NBWBNVMzJleA uid for this business is not in trainset
d4qwVw4PcN-_2mK2o1Ro1g True
d4qwVw4PcN-_2mK2o1Ro1g uid for this business is not in trainset
QARy4UUOQogMjDp9jP-YVg True
QARy4UUOQogMjDp9jP-YVg uid for this business is not in trainset
poSV39UqEg-gpESXafS9-g True
poSV39UqEg-gpESXafS9-g uid for this business is not in trainset
ujOXP2aMdYbUl4qNJrLypg True
ujOXP2aMdYbUl4qNJrLypg uid for this business is not in trainset
hcFSc0OHgZJybnjQBrL_8Q True
hcFSc0OHgZJybnjQBrL_8Q uid for this business is not 

4IGaWH9jUYMtP2uHIFEqFQ True
4IGaWH9jUYMtP2uHIFEqFQ uid for this business is not in trainset
HkHTdTvzbn-bmeQv_-2u0Q True
HkHTdTvzbn-bmeQv_-2u0Q uid for this business is not in trainset
0CElIQnJdQYF2bmsAiWczg True
0CElIQnJdQYF2bmsAiWczg uid for this business is not in trainset
He9Z_YfS6fucxkDTCVPv1Q True
He9Z_YfS6fucxkDTCVPv1Q uid for this business is not in trainset
s0QR_loWfMC2-ihWaKDT-Q True
s0QR_loWfMC2-ihWaKDT-Q uid for this business is not in trainset
lGy7jJ9oVZUJNkmZj7ewgA True
lGy7jJ9oVZUJNkmZj7ewgA uid for this business is not in trainset
0NhylBPrcpXUXZmqZL4T7w True
0NhylBPrcpXUXZmqZL4T7w uid for this business is not in trainset
GI-CAiZ_Gg3h21PwrANB4Q True
GI-CAiZ_Gg3h21PwrANB4Q uid for this business is not in trainset
u_vPjx925UPEG9DFOAAvFQ True
u_vPjx925UPEG9DFOAAvFQ uid for this business is not in trainset
62HlThvB9Rm2QFu_OdXe7w True
62HlThvB9Rm2QFu_OdXe7w uid for this business is not in trainset
eYJcI9cI6-kJ2NQpYdHcLw True
eYJcI9cI6-kJ2NQpYdHcLw uid for this business is not 

kBJfBVaXJtPEKLTov1hJdQ uid for this business is not in trainset
2pD9wZWXDNsZf_MXd8rQtg True
hIUKufhwR6Ifn7bi0-phLA True
hIUKufhwR6Ifn7bi0-phLA uid for this business is not in trainset
FCP5hYaTtn6dkpmZ_NTvEw True
FCP5hYaTtn6dkpmZ_NTvEw uid for this business is not in trainset
_fzHgewuVnNhL6a9n-sY4w True
_fzHgewuVnNhL6a9n-sY4w uid for this business is not in trainset
uLyeGUKKi_hDlnhEEqsDPA True
uLyeGUKKi_hDlnhEEqsDPA uid for this business is not in trainset
NW366Ota9w52nYofW99u1g True
NW366Ota9w52nYofW99u1g uid for this business is not in trainset
dvbcUnKv2awsIxog7dO4vw True
dvbcUnKv2awsIxog7dO4vw uid for this business is not in trainset
rcaPajgKOJC2vo_l3xa42A True
rcaPajgKOJC2vo_l3xa42A uid for this business is not in trainset
yNPh5SO-7wr8HPpVCDPbXQ True
yNPh5SO-7wr8HPpVCDPbXQ uid for this business is not in trainset
NFm869_w6cvVaWaNpAzjeA True
NFm869_w6cvVaWaNpAzjeA uid for this business is not in trainset
v4qyBRpTBvOO55M4IAqNNg True
v4qyBRpTBvOO55M4IAqNNg uid for this business is not 

P8IsTiHq5Hesa6UPL604ww True
P8IsTiHq5Hesa6UPL604ww uid for this business is not in trainset
-a857YYdjzgOdOjFFRsRXQ True
-a857YYdjzgOdOjFFRsRXQ uid for this business is not in trainset
zJGtD3y-pAIGNId4codEEg True
zJGtD3y-pAIGNId4codEEg uid for this business is not in trainset
qAHjfQqGJFClh4wHpbzHSA True
qAHjfQqGJFClh4wHpbzHSA uid for this business is not in trainset
N8jmyJCk-LjO9o1K9UjxNA True
N8jmyJCk-LjO9o1K9UjxNA uid for this business is not in trainset
UCpUOtvqR-NBWBNVMzJleA True
UCpUOtvqR-NBWBNVMzJleA uid for this business is not in trainset
UCpUOtvqR-NBWBNVMzJleA True
UCpUOtvqR-NBWBNVMzJleA uid for this business is not in trainset
0kgib8EeCvEerG1R4T1okg True
e-P17ZVKGvnBT58pqtxQeA True
e-P17ZVKGvnBT58pqtxQeA uid for this business is not in trainset
7LjFXVB7ry4V9dJf4CrWuA True
IZKjHtCuRlbhyD1WtGd8uA True
IZKjHtCuRlbhyD1WtGd8uA uid for this business is not in trainset
KUOa1acSFn6DkO9jp-sn6A True
KUOa1acSFn6DkO9jp-sn6A uid for this business is not in trainset
ZuNJelvkJD4wsXNBRg6t5w T

...

vKKSF1_1JNnRMaoNPQb5Lw uid for this business is not in trainset
vKKSF1_1JNnRMaoNPQb5Lw True
vKKSF1_1JNnRMaoNPQb5Lw uid for this business is not in trainset
vKKSF1_1JNnRMaoNPQb5Lw True
vKKSF1_1JNnRMaoNPQb5Lw uid for this business is not in trainset
vKKSF1_1JNnRMaoNPQb5Lw True
vKKSF1_1JNnRMaoNPQb5Lw uid for this business is not in trainset
vKKSF1_1JNnRMaoNPQb5Lw True
vKKSF1_1JNnRMaoNPQb5Lw uid for this business is not in trainset
vKKSF1_1JNnRMaoNPQb5Lw True
vKKSF1_1JNnRMaoNPQb5Lw uid for this business is not in trainset
vKKSF1_1JNnRMaoNPQb5Lw True
vKKSF1_1JNnRMaoNPQb5Lw uid for this business is not in trainset
vKKSF1_1JNnRMaoNPQb5Lw True
vKKSF1_1JNnRMaoNPQb5Lw uid for this business is not in trainset
YFYC7l97Yoq-jEuqbrwlwg True
NN5WoxeBHCwyfeopeS9hJw True
NN5WoxeBHCwyfeopeS9hJw uid for this business is not in trainset
NN5WoxeBHCwyfeopeS9hJw True
NN5WoxeBHCwyfeopeS9hJw uid for this business is not in trainset
8tq28_2u3gSjqEo2sKQqBA True
E3UzOSUQyoEFohk_MeLslg True
QI0PGXbGkIdInOOBeq6ieQ T

u_vPjx925UPEG9DFOAAvFQ True
u_vPjx925UPEG9DFOAAvFQ uid for this business is not in trainset
u_vPjx925UPEG9DFOAAvFQ True
u_vPjx925UPEG9DFOAAvFQ uid for this business is not in trainset
u_vPjx925UPEG9DFOAAvFQ True
u_vPjx925UPEG9DFOAAvFQ uid for this business is not in trainset
u_vPjx925UPEG9DFOAAvFQ True
u_vPjx925UPEG9DFOAAvFQ uid for this business is not in trainset
u_vPjx925UPEG9DFOAAvFQ True
u_vPjx925UPEG9DFOAAvFQ uid for this business is not in trainset
u_vPjx925UPEG9DFOAAvFQ True
u_vPjx925UPEG9DFOAAvFQ uid for this business is not in trainset
u_vPjx925UPEG9DFOAAvFQ True
u_vPjx925UPEG9DFOAAvFQ uid for this business is not in trainset
u_vPjx925UPEG9DFOAAvFQ True
u_vPjx925UPEG9DFOAAvFQ uid for this business is not in trainset
u_vPjx925UPEG9DFOAAvFQ True
u_vPjx925UPEG9DFOAAvFQ uid for this business is not in trainset
u_vPjx925UPEG9DFOAAvFQ True
u_vPjx925UPEG9DFOAAvFQ uid for this business is not in trainset
u_vPjx925UPEG9DFOAAvFQ True
u_vPjx925UPEG9DFOAAvFQ uid for this business is not 

yvPkA1ZXz1M59G6e64jZGw True
yvPkA1ZXz1M59G6e64jZGw uid for this business is not in trainset
yvPkA1ZXz1M59G6e64jZGw True
yvPkA1ZXz1M59G6e64jZGw uid for this business is not in trainset
yvPkA1ZXz1M59G6e64jZGw True
yvPkA1ZXz1M59G6e64jZGw uid for this business is not in trainset
eMCxf71sTckRZw4GVGwtrw True
Zju9aNvftYAJULT3LGsZzA True
Zju9aNvftYAJULT3LGsZzA uid for this business is not in trainset
5xX1Eb4fBoUr53OJkgRHRQ True
5xX1Eb4fBoUr53OJkgRHRQ uid for this business is not in trainset
CotnZvwxCubcWqsG2hK0gw True
260JIedrJp04Y5HBXsmAVg True
5Hcj2s3h-G9Qivyqe9pF1Q True
5Hcj2s3h-G9Qivyqe9pF1Q uid for this business is not in trainset
LkYKDy-F1lVr1MhU7FofyA True
LkYKDy-F1lVr1MhU7FofyA uid for this business is not in trainset
LkYKDy-F1lVr1MhU7FofyA True
LkYKDy-F1lVr1MhU7FofyA uid for this business is not in trainset
LkYKDy-F1lVr1MhU7FofyA True
LkYKDy-F1lVr1MhU7FofyA uid for this business is not in trainset
YLwWi9r9S2LXzZlMG1O2FQ True
YLwWi9r9S2LXzZlMG1O2FQ uid for this business is not in train

nveeiYhDhj0WJu1JUmx9fw uid for this business is not in trainset
nveeiYhDhj0WJu1JUmx9fw True
nveeiYhDhj0WJu1JUmx9fw uid for this business is not in trainset
RWFC8Ht9gWcRc2vX57OPsA True
ibzYLK6hoXlaCCibGe4PPA True
ibzYLK6hoXlaCCibGe4PPA uid for this business is not in trainset
ibzYLK6hoXlaCCibGe4PPA True
ibzYLK6hoXlaCCibGe4PPA uid for this business is not in trainset
tWjfgVtTD5n01Cq9dFWGsA True
tWjfgVtTD5n01Cq9dFWGsA uid for this business is not in trainset
u-3exVNGKE0iJ7nnxS3BbA True
u-3exVNGKE0iJ7nnxS3BbA uid for this business is not in trainset
vaImqMx2YCw7ZTMzl1hxSw True
vaImqMx2YCw7ZTMzl1hxSw uid for this business is not in trainset
8HOZJRybEinI3krJK6NHbg True
8HOZJRybEinI3krJK6NHbg uid for this business is not in trainset
5E_1xb3o97ZWQJiZJMVRmQ True
5E_1xb3o97ZWQJiZJMVRmQ uid for this business is not in trainset
5E_1xb3o97ZWQJiZJMVRmQ True
5E_1xb3o97ZWQJiZJMVRmQ uid for this business is not in trainset
tSwiWYoDBZajtwy3A_ooKg True
0fW77t7I2673JY27JWC-yw True
0fW77t7I2673JY27JWC-yw u

KGGiuMVEZE8Otj5qSAO45g True
KGGiuMVEZE8Otj5qSAO45g uid for this business is not in trainset
MFvJC0IiJwRcRCEXwlWUfg True
MFvJC0IiJwRcRCEXwlWUfg uid for this business is not in trainset
MFvJC0IiJwRcRCEXwlWUfg True
MFvJC0IiJwRcRCEXwlWUfg uid for this business is not in trainset
MFvJC0IiJwRcRCEXwlWUfg True
MFvJC0IiJwRcRCEXwlWUfg uid for this business is not in trainset
MFvJC0IiJwRcRCEXwlWUfg True
MFvJC0IiJwRcRCEXwlWUfg uid for this business is not in trainset
46Ld9Qc9nAx_A0jwclNZiw True
46Ld9Qc9nAx_A0jwclNZiw uid for this business is not in trainset
46Ld9Qc9nAx_A0jwclNZiw True
46Ld9Qc9nAx_A0jwclNZiw uid for this business is not in trainset
46Ld9Qc9nAx_A0jwclNZiw True
46Ld9Qc9nAx_A0jwclNZiw uid for this business is not in trainset
46Ld9Qc9nAx_A0jwclNZiw True
46Ld9Qc9nAx_A0jwclNZiw uid for this business is not in trainset
46Ld9Qc9nAx_A0jwclNZiw True
46Ld9Qc9nAx_A0jwclNZiw uid for this business is not in trainset
46Ld9Qc9nAx_A0jwclNZiw True
46Ld9Qc9nAx_A0jwclNZiw uid for this business is not 

In [107]:
len(uid_not_bid), len(bid_not_uid)
 

(2800, 16188)

In [116]:
u_not_b_df = None
for u, b in uid_not_bid[:20]:
    cur = union[union['user_id'] == u ]
    if u_not_b_df is not None:
        u_not_b_df = pd.concat([u_not_b_df, cur])
    else:
        u_not_b_df = cur

In [118]:
b_not_u_df = None
for u, b in bid_not_uid[:20]:
    cur = union[union['business_id'] == b ]
    if b_not_u_df is not None:
        b_not_u_df = pd.concat([b_not_u_df, cur])
    else:
        b_not_u_df = cur

In [117]:
u_not_b_df.shape

(82, 3)

In [119]:
b_not_u_df.shape

(468, 3)

In [124]:
u_not_b_list = list(zip(u_not_b_df['user_id'].tolist(), u_not_b_df['business_id'].tolist(), u_not_b_df['stars'].tolist()))

In [125]:
b_not_u_list = list(zip(b_not_u_df['user_id'].tolist(), b_not_u_df['business_id'].tolist(), b_not_u_df['stars'].tolist()))

In [127]:
len(b_not_u_list), len(u_not_b_list)

(468, 82)

In [130]:
ub_predictions = user_based_recommender.test(u_not_b_list)
ib_predictions = item_based_recommender.test(u_not_b_list)

top_n_ub = get_top_n(ub_predictions, n=10)
top_n_ib = get_top_n(ib_predictions, n=10)



# Print the recommended items for each user
for uid, user_ratings in top_n_ub.items():
    for i, (iid, score) in enumerate(user_ratings):
        print('{} {} {} {}'.format(uid, iid, score,top_n_ib[uid][i][1]))  
             

dIIKEfOgo0KqUfGQvGikPg QeGkAHAKUT-TkyPUsl2PIg 4.133333333333334 4.7004761904761905
dIIKEfOgo0KqUfGQvGikPg QeGkAHAKUT-TkyPUsl2PIg 4.133333333333334 4.7004761904761905
dIIKEfOgo0KqUfGQvGikPg QeGkAHAKUT-TkyPUsl2PIg 4.133333333333334 4.7004761904761905
dIIKEfOgo0KqUfGQvGikPg QeGkAHAKUT-TkyPUsl2PIg 4.133333333333334 4.7004761904761905
dIIKEfOgo0KqUfGQvGikPg QeGkAHAKUT-TkyPUsl2PIg 4.133333333333334 4.7004761904761905
dIIKEfOgo0KqUfGQvGikPg zJGtD3y-pAIGNId4codEEg 3.9326923076923075 4.09021978021978
dIIKEfOgo0KqUfGQvGikPg zJGtD3y-pAIGNId4codEEg 3.9326923076923075 4.09021978021978
dIIKEfOgo0KqUfGQvGikPg zJGtD3y-pAIGNId4codEEg 3.9326923076923075 4.09021978021978
dIIKEfOgo0KqUfGQvGikPg zJGtD3y-pAIGNId4codEEg 3.9326923076923075 4.09021978021978
dIIKEfOgo0KqUfGQvGikPg zJGtD3y-pAIGNId4codEEg 3.9326923076923075 4.09021978021978
JnPIjvC0cmooNDfsa9BmXg Cy9LmQrY9rWwaNTSqxQn6A 4.0 4.333333333333333
JnPIjvC0cmooNDfsa9BmXg Cy9LmQrY9rWwaNTSqxQn6A 4.0 4.333333333333333
JnPIjvC0cmooNDfsa9BmXg Cy9LmQrY9rWwaNTS

In [132]:
ub_predictions = user_based_recommender.test(b_not_u_list)
ib_predictions = item_based_recommender.test(b_not_u_list)

top_n_ub = get_top_n(ub_predictions, n=10)
top_n_ib = get_top_n(ib_predictions, n=10)



# Print the recommended items for each user
for uid, user_ratings in top_n_ub.items():
    for i, (iid, score) in enumerate(user_ratings):
        print('{} {} {} {}'.format(uid, iid, score,top_n_ib[uid][i][1]))  
             
#         print('\t', iid, top_n_ib[uid][i][0])


q-v8elVPvKz0KvK69QSj1Q fgskuH5aQq0ROHm9zst_0g 4.0 4.0
LOi1qbpl2hTP0cwSDDuyBQ fgskuH5aQq0ROHm9zst_0g 3.0 3.0
T2h6OlazEQRU9y4j1AlgIw fgskuH5aQq0ROHm9zst_0g 2.0 2.0
ex4XYmJauI3kXvR3NVNckA fgskuH5aQq0ROHm9zst_0g 1 1
KTeIV-wI3-eRXBm8qr5Y2A fgskuH5aQq0ROHm9zst_0g 4.0 4.0
ikqRpHQyIWfT4AsooJFOwg fgskuH5aQq0ROHm9zst_0g 3.0 3.0
EB11t7lYJYQO7h9mSr26tA fgskuH5aQq0ROHm9zst_0g 5 5
Fta-vmTJYKGeVyyhm9D6Vw yNPh5SO-7wr8HPpVCDPbXQ 5 5
vnWKs2dgqDuR_yNI5qJG5g yNPh5SO-7wr8HPpVCDPbXQ 4.0 4.0
PkRZYgz-Oq1dnjbRayIzwA yNPh5SO-7wr8HPpVCDPbXQ 4.0 4.0
V-BbqKqO8anwplGRx9Q5aQ yNPh5SO-7wr8HPpVCDPbXQ 3.0 2.89379945799458
sOudRDE_R3VYni0rEYFkSA yNPh5SO-7wr8HPpVCDPbXQ 3.0 3.0
h_GbSIX2fG1ejWfy_ufvWw yNPh5SO-7wr8HPpVCDPbXQ 5 5
ucj0RT3ZsWHFXQssLq-wmQ yNPh5SO-7wr8HPpVCDPbXQ 3.5 3.516629711751663
8SNyzzb2nW730niBs4iZ3w yNPh5SO-7wr8HPpVCDPbXQ 1 1
EnD0j28OHWrlDNqKhZpbiQ yNPh5SO-7wr8HPpVCDPbXQ 3.0 3.0
cM5nBvgS1Lmmto_dCfwY3Q yNPh5SO-7wr8HPpVCDPbXQ 3.0 3.9939024390243905
OqX5J5l9aPHzxYZ2EAD1YQ yNPh5SO-7wr8HPpVCDPbXQ 3.5 3.64205058

In [135]:
from surprise import dump

dump.dump('/files/user_based_recommender_nn', algo=user_based_recommender)


In [136]:
dump.dump('/files/item_based_recommender_nn', algo=item_based_recommender)
